# Description

 Bla bla bla
 
 ## 1. Libraries
 
We are importing all the required libraries for the project 

In [1]:
import pylab as pl #plot package
import pandas as pd #powerful dataframe package
import numpy as np # math package
import os #file management package
import sqlite3 # Database management package
import requests #For downloading files
from zipfile import ZipFile 
import re # for filtering text
import numpy as np # For pretty printing
import glob

## 2. Data preparation

In this step we are setting up data to be analysed.

### 2.1 Directory management 

We are creating a directory named data in the root directory.

In [2]:

# We check if we are in the data folder 
current_dir=os.getcwd()
if 'data' in current_dir:
    os.chdir('..')

# We are setting the home directory to the Jupyter Root directory
HOME_DIR=os.getcwd()

# We are saving the location of the data directory                       
DATA_DIR=HOME_DIR+'/data'

# We check if the directory exists, if not,                        
try:
    os.makedirs(DATA_DIR)
except FileExistsError :
        print("Directory {} already exists".format(DATA_DIR))
os.chdir(DATA_DIR)
print("Changing directory to {}".format(DATA_DIR))

Directory /home/jovyan/data already exists
Changing directory to /home/jovyan/data


### 2.2 Database setup

We are setting up a database in order to 

In [3]:
# Creating or connecting to the data sqlite db and setting the cursor there

database_connection = sqlite3.connect('ImportedData.db')
database_cursor = database_connection.cursor()

In [4]:
# Creating a table that traks downloaded files so we don't have to import them again

database_cursor.execute("CREATE TABLE IF NOT EXISTS file_data(file_name TEXT UNIQUE)")
database_connection.commit()
pd.read_sql_query("Select * from file_data",database_connection)

,file_name


### 2.3 Querying the Citibike data 

We are checking the data available on the citibike website. This is hosted on Amazon AwS in the tripdata Bucket.

What we do is we fetch the XML from s3, we parse it for files that are marked down inbetween `<Key>`

In [5]:
# Fetching a file list of all available files on the Citybike data server

s3_url = "https://s3.amazonaws.com/tripdata"

s3_xml_file = requests.get(s3_url).text

files_on_s3 = re.findall (r'<Key>(.*?zip)</Key>', s3_xml_file)

print ('We have found the following files on S3 ')

print ("\n".join(files_on_s3))   


We have found the following files on S3 
201306-citibike-tripdata.zip
201307-201402-citibike-tripdata.zip
201307-citibike-tripdata.zip
201308-citibike-tripdata.zip
201309-citibike-tripdata.zip
201310-citibike-tripdata.zip
201311-citibike-tripdata.zip
201312-citibike-tripdata.zip
201401-citibike-tripdata.zip
201402-citibike-tripdata.zip
201403-citibike-tripdata.zip
201404-citibike-tripdata.zip
201405-citibike-tripdata.zip
201406-citibike-tripdata.zip
201407-citibike-tripdata.zip
201408-citibike-tripdata.zip
201409-citibike-tripdata.zip
201410-citibike-tripdata.zip
201411-citibike-tripdata.zip
201412-citibike-tripdata.zip
201501-citibike-tripdata.zip
201502-citibike-tripdata.zip
201503-citibike-tripdata.zip
201504-citibike-tripdata.zip
201505-citibike-tripdata.zip
201506-citibike-tripdata.zip
201507-citibike-tripdata.zip
201508-citibike-tripdata.zip
201509-citibike-tripdata.zip
201510-citibike-tripdata.zip
201511-citibike-tripdata.zip
201512-citibike-tripdata.zip
201601-citibike-tripdata

### 2.4 Setup methods for file management

Here we define all the methods we will use when adding data into the database. We have 5 methods one for downloading, one for unzipping, one for checking if the file exists in the database, one for adding the file in the database, and a cleanup 

In [ ]:
def download_file(filename):
    delete_files_ending_with('zip')
    download_url = s3_url + '/' + filename
    print("Downloading file: " + download_url)
    file_data = requests.get(download_url)
    open(filename, 'wb').write(file_data.content)
    
def unzipping_file(filename):
    file = ZipFile(filename,'r')
    print("Extracting... ")
    file.extractall()
    unzipped_file = glob.glob('*.csv')
    print("Extracted " + unzipped_file[0] + " from " + filename + ". Deleting archive to save space...")
    delete_files_ending_with('zip')
    return unzipped_file[0]

def delete_files_ending_with(extension):
    for file in glob.glob('*'+ extension):
            os.remove(file)

def file_has_been_added_to_the_database(filename):
    database_cursor.execute("SELECT file_name FROM file_data WHERE file_name = ?",(filename,))
    if database_cursor.fetchone() is None:
        return False
    else:
        return True

def save_file_name_into_database(filename):
    database_cursor.execute("INSERT INTO file_data (file_name) VALUES(?)",(filename,))
    database_connection.commit()

def import_data_from_file_into_database(filename):
    file_data = pd.read_csv(extracted_file, names=['tripduration','starttime','stoptime','start station id','start station name',
                                      'start station latitude', 'start station longitude', 'end station id',
                                      'end station name', 'end station latitude', 'end station longitude', 'bikeid',
                                      'usertype', 'birth year', 'gender'], header=None, skiprows=[0])
    file_data.to_sql('trip_data', database_connection, if_exists='replace', index = False )
    database_connection.commit()
    print("Adding data from "+filename+" to the database")
    
    

3. We are scanning our data directory and saving the files

In [ ]:
file_count = 0

for file in files_on_s3:
    if not file_has_been_added_to_the_database(file):
        try:
            download_file(file)
            extracted_file = unzipping_file(file)
            save_file_name_into_database(file)
            import_data_from_file_into_database(extracted_file)
            delete_files_ending_with('csv')
            file_count += 1
        except XLRDError:
            print("Close any open files and try again")

print('We imported '+files_count+' files into imported_data')

Extracting... 
Extracted 201306-citibike-tripdata.csv from 201306-citibike-tripdata.zip. Deleting archive to save space...


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Adding data from %s to the database ('201306-citibike-tripdata.csv',)
Extracting... 
Extracted 2013-09 - Citi Bike trip data.csv from 201307-201402-citibike-tripdata.zip. Deleting archive to save space...
Adding data from %s to the database ('2013-09 - Citi Bike trip data.csv',)
Extracting... 
Extracted 2013-07 - Citi Bike trip data.csv from 201307-citibike-tripdata.zip. Deleting archive to save space...
Adding data from %s to the database ('2013-07 - Citi Bike trip data.csv',)
Extracting... 
Extracted 2013-08 - Citi Bike trip data.csv from 201308-citibike-tripdata.zip. Deleting archive to save space...
Adding data from %s to the database ('2013-08 - Citi Bike trip data.csv',)
Extracting... 
Extracted 2013-09 - Citi Bike trip data.csv from 201309-citibike-tripdata.zip. Deleting archive to save space...


In [ ]:
database_cursor.execute("SELECT * FROM trip_data limit 5")
print( database_cursor.fetchall())

In [ ]:
database_cursor.execute("SELECT bikeid, SUM(stoptime - starttime),  FROM trip_data")
print( database_cursor.fetchall())